In [5]:
# ============================================================
# ANNUAL -> DAILY (répartition journalière réaliste)
# - Respect strict des % mensuels (tes paramètres)
# - Effet semaine/week-end + bruit (réalisme)
# - Lecture du CSV annuel de base
# - Génère un CSV journalier final
# ============================================================

import pandas as pd
import numpy as np

# ----------------------------
# 1) Lecture du CSV annuel
# ----------------------------
CSV_PATH = "../data/patients/patients-data-interpolated.csv"  # adapte si besoin (chemin relatif/absolu)
df_annual = pd.read_csv(CSV_PATH)

# ----------------------------
# 2) Paramètres mensuels (TES %)
# ----------------------------
MONTH_PCT = {
    1: 15,
    2: 10,
    3: 7,
    4: 6,
    5: 5,
    6: 6,
    7: 13,
    8: 13,
    9: 6,
    10: 5,
    11: 4,
    12: 10,
}
if sum(MONTH_PCT.values()) != 100:
    raise ValueError(f"Les pourcentages mensuels doivent faire 100. Total={sum(MONTH_PCT.values())}")

# ----------------------------
# 2b) Arrondi par unité (nombre de décimales)
# ----------------------------
ROUNDING_BY_UNIT = {
    "%": 2,
    "%/jour": 0,
    "patients": 0,
    "patients/jour": 0,
    "ans": 0,
    "ans/jour": 0,
    "sejours": 0,
    "sejours/jour": 0,
}
DEFAULT_DECIMALS = 2  # si unité inconnue

# ----------------------------
# 3) Fonction annual -> daily
# ----------------------------
def annual_to_daily(
    annual_value: float,
    year: int,
    month_pct: dict,
    weekend_factor: float = 0.80,  # baisse week-end (logistique/linge/déchets). Pour urgences tu peux mettre >1
    weekday_factor: float = 1.05,  # petit boost en semaine
    noise_sigma: float = 0.08,      # bruit lognormal (0 => pas de bruit)
    seed: int = 42
) -> pd.DataFrame:
    """
    Répartit une valeur annuelle en journalier en respectant strictement les % mensuels,
    tout en ajoutant un effet semaine/week-end et un bruit réaliste.
    """
    rng = np.random.default_rng(seed)

    dates = pd.date_range(f"{year}-01-01", f"{year}-12-31", freq="D")
    df = pd.DataFrame({"date": dates})
    df["year"] = year
    df["month"] = df["date"].dt.month
    df["dow"] = df["date"].dt.weekday  # 0=lundi ... 6=dimanche

    values = np.zeros(len(df), dtype=float)

    for m in range(1, 13):
        idx = df.index[df["month"] == m].to_numpy()

        monthly_total = float(annual_value) * (month_pct[m] / 100.0)

        # Effet semaine / week-end
        base = np.where(df.loc[idx, "dow"].to_numpy() >= 5, weekend_factor, weekday_factor).astype(float)

        # Bruit (lognormal => positif)
        if noise_sigma > 0:
            noise = np.exp(rng.normal(loc=0.0, scale=noise_sigma, size=len(idx)))
        else:
            noise = np.ones(len(idx), dtype=float)

        weights = base * noise
        if weights.sum() == 0:
            weights = np.ones_like(weights)

        # Normalisation dans le mois => somme = monthly_total
        values[idx] = monthly_total * (weights / weights.sum())

    df["value"] = values

    # Sécurité numérique: recale exactement la somme annuelle (à cause des floats)
    df.loc[df.index[-1], "value"] += float(annual_value) - df["value"].sum()

    return df[["date", "year", "month", "dow", "value"]]

# ----------------------------
# 4) Vérification (mensuel)
# ----------------------------
def check_monthly_distribution(daily_df: pd.DataFrame, month_pct: dict) -> pd.DataFrame:
    monthly = daily_df.groupby("month")["value"].sum()
    pct_observed = (monthly / monthly.sum() * 100).round(2)
    out = pd.DataFrame({
        "pct_target": pd.Series(month_pct).sort_index(),
        "pct_observed": pct_observed
    })
    out["diff"] = (out["pct_observed"] - out["pct_target"]).round(2)
    return out

# ----------------------------
# 5) Génération journalier pour tout le CSV (PLF & CFX)
# ----------------------------
# Astuce: on adapte les paramètres selon le type d'indicateur (optionnel mais réaliste)
# - logistique (linge/déchets/restauration/colis): week-end baisse
# - urgences (si tu ajoutes ed_passages): week-end peut monter (weekend_factor > 1)
DEFAULT_PARAMS = {
    "weekend_factor": 0.80,
    "weekday_factor": 1.05,
    "noise_sigma": 0.08,
    "seed": 42,
}

# Coefficient de crise global (même logique que dans logistics.ipynb)
COEF_CRISE_GLOBAL = 1.70

all_daily = []

for i, row in df_annual.iterrows():
    year = int(row["ANNEE"])
    indic = row.get("INDICATEUR", "")
    sous_indic = row.get("SOUS-INDICATEUR", "")
    unite = row.get("UNITE", "")

    # Ici tu peux customiser selon indicateur si tu veux
    params = DEFAULT_PARAMS.copy()

    for site in ["PLF", "CFX"]:
        annual_value = row[site]

        # Skip valeurs manquantes
        if pd.isna(annual_value):
            continue

        df_d = annual_to_daily(
            annual_value=float(annual_value),
            year=year,
            month_pct=MONTH_PCT,
            **params
        )

        df_d["site_code"] = site
        df_d["indicateur"] = indic
        df_d["sous_indicateur"] = sous_indic
        df_d["unite"] = unite

        decimals = ROUNDING_BY_UNIT.get(str(unite).strip(), DEFAULT_DECIMALS)
        df_d["value"] = df_d["value"].round(decimals)
        # Valeur en mode crise (mêmes coefficients globaux que pour l'annuel)
        df_d["value_crise"] = (df_d["value"] * COEF_CRISE_GLOBAL).round(decimals)

        all_daily.append(df_d)

df_daily_all = pd.concat(all_daily, ignore_index=True)

# ----------------------------
# 6) Exemple de contrôle sur un cas
# ----------------------------
# Contrôle sur 1 combinaison (ex: première ligne, PLF)
example = df_daily_all.iloc[0:0].copy()
if len(df_daily_all) > 0:
    first = df_daily_all.iloc[0]
    example = df_daily_all[
        (df_daily_all["year"] == first["year"]) &
        (df_daily_all["site_code"] == first["site_code"]) &
        (df_daily_all["indicateur"] == first["indicateur"]) &
        (df_daily_all["sous_indicateur"] == first["sous_indicateur"])
    ]
    print("---- Vérification % mensuels (exemple) ----")
    print(check_monthly_distribution(example, MONTH_PCT))

print("\nSomme totale (toutes lignes) =", df_daily_all["value"].sum())

# Répartition par année / indicateur / sous-indicateur
summary = (
    df_daily_all
    .groupby(["year", "indicateur", "sous_indicateur"])['value']
    .sum()
    .reset_index()
    .sort_values(["year", "indicateur", "sous_indicateur"])
)
print("\nRépartition annuelle par indicateur / sous-indicateur :")
print(summary)

# ----------------------------
# 7) Export final
# ----------------------------
OUT_PATH = "../data/patients/patients_donnees_journalieres_reconstituees.csv"
df_daily_all.to_csv(OUT_PATH, index=False, encoding="utf-8")
print(f"\n✅ Fichier exporté: {OUT_PATH}")

# Tu peux afficher un aperçu (10 premières lignes)
df_daily_all.head(10)

---- Vérification % mensuels (exemple) ----
    pct_target  pct_observed  diff
1           15         14.96 -0.04
2           10          9.98 -0.02
3            7          7.00  0.00
4            6          6.02  0.02
5            5          5.03  0.03
6            6          5.99 -0.01
7           13         12.94 -0.06
8           13         13.02  0.02
9            6          6.04  0.04
10           5          5.01  0.01
11           4          3.94 -0.06
12          10         10.06  0.06

Somme totale (toutes lignes) = 752345.17

Répartition annuelle par indicateur / sous-indicateur :
     year                indicateur                       sous_indicateur  \
0    2011  Causes d'hopitalisations    Pathologies cancéreuses - Nouveaux   
1    2011  Causes d'hopitalisations     Pathologies cancéreuses - Sejours   
2    2011  Causes d'hopitalisations       Pathologies cancéreuses - Total   
3    2011      Origine géographique  Autres départements et DOM TOM - MCO   
4    2011      Or

,date,year,month,dow,value,site_code,indicateur,sous_indicateur,unite,value_crise
0,2011-01-01,2011,1,5,16.0,PLF,Causes d'hopitalisations,Pathologies cancéreuses - Nouveaux,patients,27.0
1,2011-01-02,2011,1,6,14.0,PLF,Causes d'hopitalisations,Pathologies cancéreuses - Nouveaux,patients,24.0
2,2011-01-03,2011,1,0,22.0,PLF,Causes d'hopitalisations,Pathologies cancéreuses - Nouveaux,patients,37.0
3,2011-01-04,2011,1,1,22.0,PLF,Causes d'hopitalisations,Pathologies cancéreuses - Nouveaux,patients,37.0
4,2011-01-05,2011,1,2,18.0,PLF,Causes d'hopitalisations,Pathologies cancéreuses - Nouveaux,patients,31.0
5,2011-01-06,2011,1,3,19.0,PLF,Causes d'hopitalisations,Pathologies cancéreuses - Nouveaux,patients,32.0
6,2011-01-07,2011,1,4,21.0,PLF,Causes d'hopitalisations,Pathologies cancéreuses - Nouveaux,patients,36.0
7,2011-01-08,2011,1,5,15.0,PLF,Causes d'hopitalisations,Pathologies cancéreuses - Nouveaux,patients,26.0
8,2011-01-09,2011,1,6,16.0,PLF,Causes d'hopitalisations,Pathologies cancéreuses - Nouveaux,patients,27.0
9,2011-01-10,2011,1,0,19.0,PLF,Causes d'hopitalisations,Pathologies cancéreuses - Nouveaux,patients,32.0


In [7]:
df_daily_all["indicateur"].value_counts()
df_daily_all.groupby(["site_code", "indicateur"]).size()

site_code  indicateur              
CFX        Causes d'hopitalisations     6576
           Origine géographique        43840
           Profil patients              9864
           Urgences                     1096
PLF        Causes d'hopitalisations     6576
           Origine géographique        43840
           Profil patients              9864
           Urgences                     3288
dtype: int64

In [8]:
# ============================
# Graph: Déchets / Cartons (journalier)
# - Charge le CSV journalier reconstitué
# - Filtre Déchets + Cartons
# - Choisit un site (PLF/CFX) et une année
# - Trace la courbe + moyenne mobile 7 jours
# ============================

import pandas as pd
import matplotlib.pyplot as plt

# 1) Chemin vers TON fichier journalier généré
DAILY_CSV_PATH =  "../data/patients/patients_donnees_journalieres_reconstituees.csv"  # adapte si besoin

df = pd.read_csv(DAILY_CSV_PATH)

# 2) Parse de la date
df["date"] = pd.to_datetime(df["date"])

# 3) Paramètres du graphe
SITE = "PLF"         # "PLF" ou "CFX"
YEAR = 2012          # année à visualiser

# 4) Filtre: 
mask = (
    (df["site_code"] == SITE) &
    (df["year"] == YEAR) &
    (df["indicateur"].str.strip() == "Urgences") &
    (df["sous_indicateur"].str.strip().str.lower() == "Passages")
)

cartons = df.loc[mask, ["date", "value", "unite"]].sort_values("date").copy()

if cartons.empty:
    print("❌ Aucune donnée trouvée pour Déchets / Cartons avec ces filtres.")
    print("➡️ Vérifie les valeurs exactes de 'indicateur' et 'sous_indicateur' avec :")
    print('   df[df["indicateur"]=="Déchets"]["sous_indicateur"].unique()')
else:
    unit = cartons["unite"].iloc[0] if "unite" in cartons.columns and len(cartons) > 0 else ""
    cartons["mm7"] = cartons["value"].rolling(7, min_periods=1).mean()

    # 5) Plot
    plt.figure(figsize=(14, 5))
    plt.plot(cartons["date"], cartons["value"], label="Journalier")
    plt.plot(cartons["date"], cartons["mm7"], label="Moyenne mobile 7j")
    plt.title(f"Déchets - Cartons ({SITE}) - {YEAR}")
    plt.xlabel("Date")
    plt.ylabel(f"Volume ({unit})" if unit else "Volume")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    # (Optionnel) Aperçu
    display(cartons.head(10))

❌ Aucune donnée trouvée pour Déchets / Cartons avec ces filtres.
➡️ Vérifie les valeurs exactes de 'indicateur' et 'sous_indicateur' avec :
   df[df["indicateur"]=="Déchets"]["sous_indicateur"].unique()
